# Understanding Messages in LangChain 1.0

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EnkrateiaLucca/oreilly_live_training_getting_started_with_langchain/blob/main/notebooks/1.2-messages-deep-dive.ipynb)

Messages are the fundamental unit of communication in LangChain 1.0. This notebook explores:

- **Message types**: HumanMessage, AIMessage, SystemMessage, ToolMessage
- **Input flexibility**: Multiple ways to send messages
- **Message metadata**: Token usage, response info, content blocks
- **Debugging**: Using `.pretty_print()` for inspection

In [ ]:
# Install dependencies (uncomment if running in Colab)
# !pip install langchain>=1.0.0 langchain-openai>=0.2.0

In [ ]:
import os
from getpass import getpass

# Set API key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

## 1. Message Types Overview

LangChain provides several message types, each serving a specific purpose in the conversation:

| Type | Purpose | When to Use |
|------|---------|-------------|
| `SystemMessage` | Set behavior/context | Once at start of conversation |
| `HumanMessage` | User input | Every user turn |
| `AIMessage` | Model response | Returned by the model |
| `ToolMessage` | Tool execution results | After tool calls |

In [ ]:
from langchain_core.messages import (
    HumanMessage,
    AIMessage,
    SystemMessage,
    ToolMessage,
)

# Create different message types
system_msg = SystemMessage(content="You are a helpful coding assistant.")
human_msg = HumanMessage(content="What is a decorator in Python?")
ai_msg = AIMessage(content="A decorator is a function that wraps another function...")

print(f"System: {system_msg.type} -> {system_msg.content[:50]}...")
print(f"Human: {human_msg.type} -> {human_msg.content[:50]}...")
print(f"AI: {ai_msg.type} -> {ai_msg.content[:50]}...")

## 2. Input Format Flexibility

LangChain 1.0 accepts messages in multiple formats - choose what's most convenient for your use case:

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Format 1: Simple string (converted to HumanMessage)
response1 = llm.invoke("What is 2 + 2?")
print("Format 1 (string):")
print(f"  Input type: str")
print(f"  Response: {response1.content}\n")

# Format 2: Dictionary format (OpenAI-style)
response2 = llm.invoke([{"role": "user", "content": "What is 2 + 2?"}])
print("Format 2 (dict):")
print(f"  Input type: dict")
print(f"  Response: {response2.content}\n")

# Format 3: Explicit LangChain message objects
response3 = llm.invoke([HumanMessage(content="What is 2 + 2?")])
print("Format 3 (HumanMessage):")
print(f"  Input type: HumanMessage")
print(f"  Response: {response3.content}")

### Multi-turn Conversations

Build up conversation history by including previous messages:

In [ ]:
# Multi-turn conversation
messages = [
    SystemMessage(content="You are a Python tutor. Give concise answers."),
    HumanMessage(content="What is a list comprehension?"),
    AIMessage(content="A list comprehension is a concise way to create lists: [x*2 for x in range(5)]"),
    HumanMessage(content="Can I add conditions to it?"),
]

response = llm.invoke(messages)
print(response.content)

## 3. Message Metadata

Messages contain rich metadata that's useful for debugging, cost tracking, and understanding model behavior.

In [ ]:
# Get a response with full metadata
response = llm.invoke("Explain async/await in Python in 2 sentences.")

print("=== Message Properties ===")
print(f"Content: {response.content}\n")
print(f"Type: {response.type}")
print(f"ID: {response.id}")

In [ ]:
# Token usage metadata - useful for cost tracking
print("=== Token Usage ===")
if response.usage_metadata:
    print(f"Input tokens: {response.usage_metadata.get('input_tokens', 'N/A')}")
    print(f"Output tokens: {response.usage_metadata.get('output_tokens', 'N/A')}")
    print(f"Total tokens: {response.usage_metadata.get('total_tokens', 'N/A')}")
else:
    print("No usage metadata available")

In [ ]:
# Response metadata - model info and settings
print("=== Response Metadata ===")
if response.response_metadata:
    print(f"Model: {response.response_metadata.get('model_name', 'N/A')}")
    print(f"Finish reason: {response.response_metadata.get('finish_reason', 'N/A')}")
    print(f"System fingerprint: {response.response_metadata.get('system_fingerprint', 'N/A')[:20]}...")

### Cost Calculation Example

Use token metadata to calculate API costs:

In [ ]:
# GPT-4o-mini pricing (as of late 2024)
INPUT_COST_PER_1M = 0.15  # $0.15 per 1M input tokens
OUTPUT_COST_PER_1M = 0.60  # $0.60 per 1M output tokens

def calculate_cost(response):
    """Calculate cost from response metadata."""
    if not response.usage_metadata:
        return None
    
    input_tokens = response.usage_metadata.get('input_tokens', 0)
    output_tokens = response.usage_metadata.get('output_tokens', 0)
    
    input_cost = (input_tokens / 1_000_000) * INPUT_COST_PER_1M
    output_cost = (output_tokens / 1_000_000) * OUTPUT_COST_PER_1M
    
    return {
        'input_cost': input_cost,
        'output_cost': output_cost,
        'total_cost': input_cost + output_cost
    }

cost = calculate_cost(response)
if cost:
    print(f"Estimated cost: ${cost['total_cost']:.6f}")

## 4. Debugging with pretty_print()

The `.pretty_print()` method provides a formatted view of messages - invaluable for debugging complex chains:

In [ ]:
# Pretty print a single message
print("=== Single Message ===")
response.pretty_print()

In [ ]:
# Pretty print conversation history
conversation = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What's the capital of France?"),
    AIMessage(content="The capital of France is Paris."),
    HumanMessage(content="What's its population?"),
]

print("=== Conversation History ===")
for msg in conversation:
    msg.pretty_print()
    print()  # Add spacing

## 5. Tool Messages

When agents use tools, the results are returned as `ToolMessage` objects:

In [ ]:
from langchain_core.tools import tool

@tool
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    # Simulated response
    return f"Weather in {city}: 72°F, sunny"

# Simulate a tool call and response
tool_call_id = "call_abc123"

# This is what an AI message with tool calls looks like
ai_with_tool = AIMessage(
    content="",
    tool_calls=[{
        "id": tool_call_id,
        "name": "get_weather",
        "args": {"city": "San Francisco"}
    }]
)

# Execute the tool
result = get_weather.invoke({"city": "San Francisco"})

# Create tool message with the result
tool_msg = ToolMessage(
    content=result,
    tool_call_id=tool_call_id,
    name="get_weather"
)

print("Tool Message:")
print(f"  Type: {tool_msg.type}")
print(f"  Tool Call ID: {tool_msg.tool_call_id}")
print(f"  Content: {tool_msg.content}")

## 6. Content Blocks (Multimodal)

LangChain 1.0 supports multimodal content through content blocks:

In [ ]:
# Text and image in the same message (GPT-4 Vision)
multimodal_message = HumanMessage(
    content=[
        {"type": "text", "text": "What's in this image?"},
        {
            "type": "image_url",
            "image_url": {
                "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4c/Python_logo_51.svg/200px-Python_logo_51.svg.png"
            }
        }
    ]
)

print("Multimodal message content blocks:")
for i, block in enumerate(multimodal_message.content):
    print(f"  Block {i}: type={block.get('type')}")

In [ ]:
# Invoke with multimodal content (requires vision-capable model)
vision_llm = ChatOpenAI(model="gpt-4o-mini")  # gpt-4o-mini supports vision

response = vision_llm.invoke([multimodal_message])
print(response.content)

## 7. Practical Example: Conversation Manager

Here's a practical pattern for managing conversations with proper message handling:

In [ ]:
class ConversationManager:
    """Simple conversation manager with token tracking."""
    
    def __init__(self, system_prompt: str, model: str = "gpt-4o-mini"):
        self.llm = ChatOpenAI(model=model, temperature=0.7)
        self.messages = [SystemMessage(content=system_prompt)]
        self.total_tokens = 0
        self.total_cost = 0.0
    
    def chat(self, user_input: str) -> str:
        """Send a message and get a response."""
        # Add user message
        self.messages.append(HumanMessage(content=user_input))
        
        # Get response
        response = self.llm.invoke(self.messages)
        
        # Track tokens
        if response.usage_metadata:
            tokens = response.usage_metadata.get('total_tokens', 0)
            self.total_tokens += tokens
        
        # Add AI response to history
        self.messages.append(response)
        
        return response.content
    
    def get_stats(self) -> dict:
        """Get conversation statistics."""
        return {
            "turns": len([m for m in self.messages if isinstance(m, HumanMessage)]),
            "total_tokens": self.total_tokens,
            "message_count": len(self.messages)
        }
    
    def show_history(self):
        """Print conversation history."""
        for msg in self.messages:
            msg.pretty_print()
            print()

In [ ]:
# Use the conversation manager
conv = ConversationManager(
    system_prompt="You are a helpful Python tutor. Keep answers concise."
)

# Have a conversation
print("User: What is a generator?")
print(f"AI: {conv.chat('What is a generator?')}\n")

print("User: Show me an example")
print(f"AI: {conv.chat('Show me an example')}\n")

# Check stats
print("\n=== Conversation Stats ===")
stats = conv.get_stats()
print(f"Turns: {stats['turns']}")
print(f"Total tokens: {stats['total_tokens']}")
print(f"Messages in history: {stats['message_count']}")

## Summary

**Key Takeaways:**

1. **Message Types**: Use `SystemMessage` for behavior, `HumanMessage` for user input, `AIMessage` for responses
2. **Input Flexibility**: Strings, dicts, or explicit message objects all work
3. **Metadata**: Use `usage_metadata` for token tracking and cost calculation
4. **Debugging**: `.pretty_print()` is your friend for inspecting messages
5. **Multimodal**: Content blocks support text + images in the same message

**Next Steps:**
- Explore streaming patterns in the next notebook
- Learn how agents handle tool messages automatically